In [2]:
import pandas as pd
import numpy as np
import re
from rapidfuzz import fuzz, process
from IPython.display import FileLink

# The Task:
- identify duplicate companies and provide a description of my approach/code to solve this problem under certain rules

# Rules:
- Joint Ventures should be assigned MTAuIDs different from MTAuIDs assigned to the companies that are parts of the Joint Ventures
- Companies that have DBAs (Doing Business As) should be assigned the same MTAuID as DBAs
- Individuals that are presidents/CEOs of the companies should be assigned the same MTAuIDs as the companies.
- Companies with a Trade Name, should be assigned the same MTAuID  as the Trade Name company.
- Companies that have AKA (also known as) name, should be assigned the same MTAuiDs as AKA company.
- If a company is a division of the other company, they should be assigned the same MTAuIDs.

# Step 1: 
- Load the excel file into a pandas dataframe so that it can be manipulated using python.

In [3]:
company = pd.read_excel('Companies_SamePhone_DifferentID.xlsx') #Loads the excel file to a pandas dataframe
company

,MTAuID,MTAuID_Correct,Name,DBA Name,Phone,Address,City,Email,Website
0,MARYBACONART,NaN,MARY BACON ART,NaN,16122101599,1001 FRONT ST.,CRESCENT CITY,NaN,https://marybaconart.com/
1,MARYBACONINTERIOR,NaN,MARY BACON INTERIORS,NaN,16122101599,1001 FRONT ST,CRESCENT CITY,NaN,http://www.marybaconinteriors.com/
2,DUBOSEDESIGNGROUP,NaN,"DUBOSE DESIGN GROUP, INC.",NaN,17603538110,1065 W STATE STREET,EL CENTRO,tomdubose@sbcglobal.net,dubosedesigngroup.com
3,LCENGINEERINGCONSULTANT,NaN,"LC ENGINEERING CONSULTANTS, INC.",NaN,17603538110,1065 W STATE STREET,EL CENTRO,carloscorrales@dde-inc.net,https://lcec-inc.com/
4,ARCHERSTREET,NaN,ARCHER STREET LLC,ARCHER STREET,2022157295,4518 11TH AVENUE,LOS ANGELES,tony@archerstreet.com,https://www.archerstreet.com/
...,...,...,...,...,...,...,...,...,...
7581,SPEEDWAY46O8,NaN,SPEEDWAY #4608,WESTERN REFINING RETAIL LLC,9728280711,2435 OTAY CENTER DR,SAN DIEGO,NaN,NaN
7582,HM,NaN,H&M,H&M FASHION USA INC,9739798198,4461 CAMINO DE LA PLAZA SUITE 301,SAN YSIDRO,NaN,NaN
7583,HMFASHIONUSA,NaN,H&M FASHION USA INC,NaN,9739798198,7007 FRIARS RD SUITE 701,SAN DIEGO,NaN,NaN
7584,RECORDXPRESOFCALIFORNIA,NaN,RECORD PRESERVATION & MANAGEMT,ACCESS INFORMATION MANAGEMENT,9785047085,DBA ACCESS INFORMATION MANAGEMENT PO BOX 4837,HAYWARD,ANNIE.WONG@ACCESSCORP.COM,NaN


# Step 2:
- Fill in all the MTAuID_Correct Column with DBA (Doing Business As) name if there contains that value. If it does not then use whatever is in the MTAuID column 

In [4]:
company['MTAuID_Correct'] = company['MTAuID_Correct'].fillna(company['MTAuID']) #Fills using DBA Name
company['MTAuID_Correct'] = company['DBA Name'].combine_first(company['MTAuID_Correct']) # Fills columns using MTAuID Name because does not have DBA Name
company

,MTAuID,MTAuID_Correct,Name,DBA Name,Phone,Address,City,Email,Website
0,MARYBACONART,MARYBACONART,MARY BACON ART,NaN,16122101599,1001 FRONT ST.,CRESCENT CITY,NaN,https://marybaconart.com/
1,MARYBACONINTERIOR,MARYBACONINTERIOR,MARY BACON INTERIORS,NaN,16122101599,1001 FRONT ST,CRESCENT CITY,NaN,http://www.marybaconinteriors.com/
2,DUBOSEDESIGNGROUP,DUBOSEDESIGNGROUP,"DUBOSE DESIGN GROUP, INC.",NaN,17603538110,1065 W STATE STREET,EL CENTRO,tomdubose@sbcglobal.net,dubosedesigngroup.com
3,LCENGINEERINGCONSULTANT,LCENGINEERINGCONSULTANT,"LC ENGINEERING CONSULTANTS, INC.",NaN,17603538110,1065 W STATE STREET,EL CENTRO,carloscorrales@dde-inc.net,https://lcec-inc.com/
4,ARCHERSTREET,ARCHER STREET,ARCHER STREET LLC,ARCHER STREET,2022157295,4518 11TH AVENUE,LOS ANGELES,tony@archerstreet.com,https://www.archerstreet.com/
...,...,...,...,...,...,...,...,...,...
7581,SPEEDWAY46O8,WESTERN REFINING RETAIL LLC,SPEEDWAY #4608,WESTERN REFINING RETAIL LLC,9728280711,2435 OTAY CENTER DR,SAN DIEGO,NaN,NaN
7582,HM,H&M FASHION USA INC,H&M,H&M FASHION USA INC,9739798198,4461 CAMINO DE LA PLAZA SUITE 301,SAN YSIDRO,NaN,NaN
7583,HMFASHIONUSA,HMFASHIONUSA,H&M FASHION USA INC,NaN,9739798198,7007 FRIARS RD SUITE 701,SAN DIEGO,NaN,NaN
7584,RECORDXPRESOFCALIFORNIA,ACCESS INFORMATION MANAGEMENT,RECORD PRESERVATION & MANAGEMT,ACCESS INFORMATION MANAGEMENT,9785047085,DBA ACCESS INFORMATION MANAGEMENT PO BOX 4837,HAYWARD,ANNIE.WONG@ACCESSCORP.COM,NaN


# Step 3:
- Group together all the companies by the same phone number

In [5]:
phone_group = company.groupby('Phone').agg(list)
phone_group

,MTAuID,MTAuID_Correct,Name,DBA Name,Address,City,Email,Website
Phone,,,,,,,,
16122101599,"[MARYBACONART, MARYBACONINTERIOR]","[MARYBACONART, MARYBACONINTERIOR]","[MARY BACON ART, MARY BACON INTERIORS]","[nan, nan]","[1001 FRONT ST., 1001 FRONT ST]","[CRESCENT CITY, CRESCENT CITY]","[nan, nan]","[https://marybaconart.com/, http://www.marybac..."
17603538110,"[DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT]","[DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT]","[DUBOSE DESIGN GROUP, INC., LC ENGINEERING CON...","[nan, nan]","[1065 W STATE STREET, 1065 W STATE STREET]","[EL CENTRO, EL CENTRO]","[tomdubose@sbcglobal.net, carloscorrales@dde-i...","[dubosedesigngroup.com, https://lcec-inc.com/]"
2022157295,"[ARCHERSTREET, SIVILTECHNOLOGY]","[ARCHER STREET, SIVILTECHNOLOGY]","[ARCHER STREET LLC, SIVIL TECHNOLOGIES INC]","[ARCHER STREET, nan]","[4518 11TH AVENUE, PO BOX 561782]","[LOS ANGELES, LOS ANGELES]","[tony@archerstreet.com, tony@sivilco.com]","[https://www.archerstreet.com/, nan]"
2085960528,"[AJSURVEYING, AJSWIRESAGGING]","[AJSURVEYING, AJSWIRESAGGING]","[A J SURVEYING, INC., A J S WIRE SAGGING, INC.]","[nan, nan]","[10957 CANIS LN, 9431 DOWDY DR, SUITE 2]","[SAN DIEGO, SAN DIEGO]","[nan, nan]","[nan, nan]"
2092290735,"[MPSTRUCTURALENGINEER, NDSCONSULTING, EXCELLCO...","[MPSTRUCTURALENGINEER, NDSCONSULTING, COVENANT...","[MP STRUCTURAL ENGINEERS, NDS CONSULTING LLC, ...","[nan, nan, COVENANT SUPPLY HOUSE]","[2633 CELAYA CIR, 22320 FOOTHILL BLVD, SUITE 6...","[SAN RAMON, HAYWARD, STOCKTON]","[davis@mpstructural.com, davis@ndsconsultingll...","[http://www.mpstructural.com/, nan, http://www..."
...,...,...,...,...,...,...,...,...
9519927316,"[VONDEZ, AICON]","[VONDEZ, VONDEZ LLC]","[VONDEZ LLC, A I CONSTRUCTION]","[nan, VONDEZ LLC]","[nan, 16427 SADDLEBROOK LN]","[MORENO VALLEY, MORENO VALLEY]","[vondezllc@yahoo.com, nan]","[nan, nan]"
9708468145,"[THRIVISTER, RAZCOACHINGSVC]","[THRIVISTER, RAZCOACHINGSVC]","[THRIVISTER LLC,, RAZ COACHING SERVICES LLC]","[nan, nan]","[nan, 109 NORTH GRANITE CIRCLE]","[FOLSOM, FOLSOM]","[michelle@thrivister.com, michelle@razcoaching...","[nan, htttp://www.razcoaching.com]"
9728280711,"[SPEEDWAY545O, SPEEDWAY46O8]","[WESTERN REFINING RETAIL LLC, WESTERN REFINING...","[SPEEDWAY #5450, SPEEDWAY #4608]","[WESTERN REFINING RETAIL LLC, WESTERN REFINING...","[9490 MIRA MESA BLVD, 2435 OTAY CENTER DR]","[SAN DIEGO, SAN DIEGO]","[nan, nan]","[nan, nan]"


# Step 4:
- Looking at the MTAuID_Correct column, take out all the companies that have the same DBA name and/or MTAuID name

In [6]:
phone_group['MTAuID_Correct'] = phone_group['MTAuID_Correct'].apply(lambda x: list(set(x)))
phone_group

,MTAuID,MTAuID_Correct,Name,DBA Name,Address,City,Email,Website
Phone,,,,,,,,
16122101599,"[MARYBACONART, MARYBACONINTERIOR]","[MARYBACONINTERIOR, MARYBACONART]","[MARY BACON ART, MARY BACON INTERIORS]","[nan, nan]","[1001 FRONT ST., 1001 FRONT ST]","[CRESCENT CITY, CRESCENT CITY]","[nan, nan]","[https://marybaconart.com/, http://www.marybac..."
17603538110,"[DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT]","[DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT]","[DUBOSE DESIGN GROUP, INC., LC ENGINEERING CON...","[nan, nan]","[1065 W STATE STREET, 1065 W STATE STREET]","[EL CENTRO, EL CENTRO]","[tomdubose@sbcglobal.net, carloscorrales@dde-i...","[dubosedesigngroup.com, https://lcec-inc.com/]"
2022157295,"[ARCHERSTREET, SIVILTECHNOLOGY]","[SIVILTECHNOLOGY, ARCHER STREET]","[ARCHER STREET LLC, SIVIL TECHNOLOGIES INC]","[ARCHER STREET, nan]","[4518 11TH AVENUE, PO BOX 561782]","[LOS ANGELES, LOS ANGELES]","[tony@archerstreet.com, tony@sivilco.com]","[https://www.archerstreet.com/, nan]"
2085960528,"[AJSURVEYING, AJSWIRESAGGING]","[AJSURVEYING, AJSWIRESAGGING]","[A J SURVEYING, INC., A J S WIRE SAGGING, INC.]","[nan, nan]","[10957 CANIS LN, 9431 DOWDY DR, SUITE 2]","[SAN DIEGO, SAN DIEGO]","[nan, nan]","[nan, nan]"
2092290735,"[MPSTRUCTURALENGINEER, NDSCONSULTING, EXCELLCO...","[MPSTRUCTURALENGINEER, COVENANT SUPPLY HOUSE, ...","[MP STRUCTURAL ENGINEERS, NDS CONSULTING LLC, ...","[nan, nan, COVENANT SUPPLY HOUSE]","[2633 CELAYA CIR, 22320 FOOTHILL BLVD, SUITE 6...","[SAN RAMON, HAYWARD, STOCKTON]","[davis@mpstructural.com, davis@ndsconsultingll...","[http://www.mpstructural.com/, nan, http://www..."
...,...,...,...,...,...,...,...,...
9519927316,"[VONDEZ, AICON]","[VONDEZ, VONDEZ LLC]","[VONDEZ LLC, A I CONSTRUCTION]","[nan, VONDEZ LLC]","[nan, 16427 SADDLEBROOK LN]","[MORENO VALLEY, MORENO VALLEY]","[vondezllc@yahoo.com, nan]","[nan, nan]"
9708468145,"[THRIVISTER, RAZCOACHINGSVC]","[THRIVISTER, RAZCOACHINGSVC]","[THRIVISTER LLC,, RAZ COACHING SERVICES LLC]","[nan, nan]","[nan, 109 NORTH GRANITE CIRCLE]","[FOLSOM, FOLSOM]","[michelle@thrivister.com, michelle@razcoaching...","[nan, htttp://www.razcoaching.com]"
9728280711,"[SPEEDWAY545O, SPEEDWAY46O8]",[WESTERN REFINING RETAIL LLC],"[SPEEDWAY #5450, SPEEDWAY #4608]","[WESTERN REFINING RETAIL LLC, WESTERN REFINING...","[9490 MIRA MESA BLVD, 2435 OTAY CENTER DR]","[SAN DIEGO, SAN DIEGO]","[nan, nan]","[nan, nan]"


# Step 5:
- ### Problem: 
    There are companies that are the same but the format in which the name is spelled varies a litte. For example Vondez and Vondez LLC are the same company, but because of the format it does not distinguish the two.

- ### Goal:
    Ensure that companies that are the same company (Ex: Vondez and Vondez LLC) have the same MTAuID

- ### Solution
    Using RapidFuzz which is a python library that is most efficient for speed and low dependency requirement. It will match similar how similar each inputs are perfect for examples in which there is only a slight difference. (Ex: Vondez and Vondez LLC)

In [7]:
fuzz.partial_ratio('Vondez', 'Vondez LLC') #Here is an example. 

100.0

In [8]:
def remove_similar(company_list, threshold=78): #The threshold is set at 78 and can be increased if you want it to be stricted
    '''
    Removes any similar strings past a threshold using fuzz.partial_ratio
    Input: Takes in a list of strings 
    Output: A list of strings with similar strings removed
    '''
    unique_company = [] #The unique strings that are not similar to any other strings in the list past the threshold
    for name in company_list: # Loops through the list taking out similar strings
        if not any(fuzz.partial_ratio(name, unique) >= threshold for unique in unique_company):
            unique_company.append(name)
    return unique_company # Returns the list of strings with similar strings removed

In [9]:
phone_group['MTAuID_Unique'] = phone_group['MTAuID_Correct'].apply(remove_similar)

In [10]:
phone_group

,MTAuID,MTAuID_Correct,Name,DBA Name,Address,City,Email,Website,MTAuID_Unique
Phone,,,,,,,,,
16122101599,"[MARYBACONART, MARYBACONINTERIOR]","[MARYBACONINTERIOR, MARYBACONART]","[MARY BACON ART, MARY BACON INTERIORS]","[nan, nan]","[1001 FRONT ST., 1001 FRONT ST]","[CRESCENT CITY, CRESCENT CITY]","[nan, nan]","[https://marybaconart.com/, http://www.marybac...",[MARYBACONINTERIOR]
17603538110,"[DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT]","[DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT]","[DUBOSE DESIGN GROUP, INC., LC ENGINEERING CON...","[nan, nan]","[1065 W STATE STREET, 1065 W STATE STREET]","[EL CENTRO, EL CENTRO]","[tomdubose@sbcglobal.net, carloscorrales@dde-i...","[dubosedesigngroup.com, https://lcec-inc.com/]","[DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT]"
2022157295,"[ARCHERSTREET, SIVILTECHNOLOGY]","[SIVILTECHNOLOGY, ARCHER STREET]","[ARCHER STREET LLC, SIVIL TECHNOLOGIES INC]","[ARCHER STREET, nan]","[4518 11TH AVENUE, PO BOX 561782]","[LOS ANGELES, LOS ANGELES]","[tony@archerstreet.com, tony@sivilco.com]","[https://www.archerstreet.com/, nan]","[SIVILTECHNOLOGY, ARCHER STREET]"
2085960528,"[AJSURVEYING, AJSWIRESAGGING]","[AJSURVEYING, AJSWIRESAGGING]","[A J SURVEYING, INC., A J S WIRE SAGGING, INC.]","[nan, nan]","[10957 CANIS LN, 9431 DOWDY DR, SUITE 2]","[SAN DIEGO, SAN DIEGO]","[nan, nan]","[nan, nan]","[AJSURVEYING, AJSWIRESAGGING]"
2092290735,"[MPSTRUCTURALENGINEER, NDSCONSULTING, EXCELLCO...","[MPSTRUCTURALENGINEER, COVENANT SUPPLY HOUSE, ...","[MP STRUCTURAL ENGINEERS, NDS CONSULTING LLC, ...","[nan, nan, COVENANT SUPPLY HOUSE]","[2633 CELAYA CIR, 22320 FOOTHILL BLVD, SUITE 6...","[SAN RAMON, HAYWARD, STOCKTON]","[davis@mpstructural.com, davis@ndsconsultingll...","[http://www.mpstructural.com/, nan, http://www...","[MPSTRUCTURALENGINEER, COVENANT SUPPLY HOUSE, ..."
...,...,...,...,...,...,...,...,...,...
9519927316,"[VONDEZ, AICON]","[VONDEZ, VONDEZ LLC]","[VONDEZ LLC, A I CONSTRUCTION]","[nan, VONDEZ LLC]","[nan, 16427 SADDLEBROOK LN]","[MORENO VALLEY, MORENO VALLEY]","[vondezllc@yahoo.com, nan]","[nan, nan]",[VONDEZ]
9708468145,"[THRIVISTER, RAZCOACHINGSVC]","[THRIVISTER, RAZCOACHINGSVC]","[THRIVISTER LLC,, RAZ COACHING SERVICES LLC]","[nan, nan]","[nan, 109 NORTH GRANITE CIRCLE]","[FOLSOM, FOLSOM]","[michelle@thrivister.com, michelle@razcoaching...","[nan, htttp://www.razcoaching.com]","[THRIVISTER, RAZCOACHINGSVC]"
9728280711,"[SPEEDWAY545O, SPEEDWAY46O8]",[WESTERN REFINING RETAIL LLC],"[SPEEDWAY #5450, SPEEDWAY #4608]","[WESTERN REFINING RETAIL LLC, WESTERN REFINING...","[9490 MIRA MESA BLVD, 2435 OTAY CENTER DR]","[SAN DIEGO, SAN DIEGO]","[nan, nan]","[nan, nan]",[WESTERN REFINING RETAIL LLC]


# Step 6:
- Clean up the DataFrame so it easier to look at

In [11]:
reordered_df = phone_group[['MTAuID_Unique', 'MTAuID_Correct', 'Name', 'DBA Name', 'Address', 'City', 'Email', 'Website']]
reordered_df

,MTAuID_Unique,MTAuID_Correct,Name,DBA Name,Address,City,Email,Website
Phone,,,,,,,,
16122101599,[MARYBACONINTERIOR],"[MARYBACONINTERIOR, MARYBACONART]","[MARY BACON ART, MARY BACON INTERIORS]","[nan, nan]","[1001 FRONT ST., 1001 FRONT ST]","[CRESCENT CITY, CRESCENT CITY]","[nan, nan]","[https://marybaconart.com/, http://www.marybac..."
17603538110,"[DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT]","[DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT]","[DUBOSE DESIGN GROUP, INC., LC ENGINEERING CON...","[nan, nan]","[1065 W STATE STREET, 1065 W STATE STREET]","[EL CENTRO, EL CENTRO]","[tomdubose@sbcglobal.net, carloscorrales@dde-i...","[dubosedesigngroup.com, https://lcec-inc.com/]"
2022157295,"[SIVILTECHNOLOGY, ARCHER STREET]","[SIVILTECHNOLOGY, ARCHER STREET]","[ARCHER STREET LLC, SIVIL TECHNOLOGIES INC]","[ARCHER STREET, nan]","[4518 11TH AVENUE, PO BOX 561782]","[LOS ANGELES, LOS ANGELES]","[tony@archerstreet.com, tony@sivilco.com]","[https://www.archerstreet.com/, nan]"
2085960528,"[AJSURVEYING, AJSWIRESAGGING]","[AJSURVEYING, AJSWIRESAGGING]","[A J SURVEYING, INC., A J S WIRE SAGGING, INC.]","[nan, nan]","[10957 CANIS LN, 9431 DOWDY DR, SUITE 2]","[SAN DIEGO, SAN DIEGO]","[nan, nan]","[nan, nan]"
2092290735,"[MPSTRUCTURALENGINEER, COVENANT SUPPLY HOUSE, ...","[MPSTRUCTURALENGINEER, COVENANT SUPPLY HOUSE, ...","[MP STRUCTURAL ENGINEERS, NDS CONSULTING LLC, ...","[nan, nan, COVENANT SUPPLY HOUSE]","[2633 CELAYA CIR, 22320 FOOTHILL BLVD, SUITE 6...","[SAN RAMON, HAYWARD, STOCKTON]","[davis@mpstructural.com, davis@ndsconsultingll...","[http://www.mpstructural.com/, nan, http://www..."
...,...,...,...,...,...,...,...,...
9519927316,[VONDEZ],"[VONDEZ, VONDEZ LLC]","[VONDEZ LLC, A I CONSTRUCTION]","[nan, VONDEZ LLC]","[nan, 16427 SADDLEBROOK LN]","[MORENO VALLEY, MORENO VALLEY]","[vondezllc@yahoo.com, nan]","[nan, nan]"
9708468145,"[THRIVISTER, RAZCOACHINGSVC]","[THRIVISTER, RAZCOACHINGSVC]","[THRIVISTER LLC,, RAZ COACHING SERVICES LLC]","[nan, nan]","[nan, 109 NORTH GRANITE CIRCLE]","[FOLSOM, FOLSOM]","[michelle@thrivister.com, michelle@razcoaching...","[nan, htttp://www.razcoaching.com]"
9728280711,[WESTERN REFINING RETAIL LLC],[WESTERN REFINING RETAIL LLC],"[SPEEDWAY #5450, SPEEDWAY #4608]","[WESTERN REFINING RETAIL LLC, WESTERN REFINING...","[9490 MIRA MESA BLVD, 2435 OTAY CENTER DR]","[SAN DIEGO, SAN DIEGO]","[nan, nan]","[nan, nan]"


In [12]:
reordered_df= reordered_df.applymap(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
reordered_df

/var/folders/35/0qd27pkn3dl08t6w6xw_cm_00000gn/T/ipykernel_48643/4250426711.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  reordered_df= reordered_df.applymap(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)


,MTAuID_Unique,MTAuID_Correct,Name,DBA Name,Address,City,Email,Website
Phone,,,,,,,,
16122101599,MARYBACONINTERIOR,"MARYBACONINTERIOR, MARYBACONART","MARY BACON ART, MARY BACON INTERIORS","nan, nan","1001 FRONT ST., 1001 FRONT ST","CRESCENT CITY, CRESCENT CITY","nan, nan","https://marybaconart.com/, http://www.marybaco..."
17603538110,"DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT","DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT","DUBOSE DESIGN GROUP, INC., LC ENGINEERING CONS...","nan, nan","1065 W STATE STREET, 1065 W STATE STREET","EL CENTRO, EL CENTRO","tomdubose@sbcglobal.net, carloscorrales@dde-in...","dubosedesigngroup.com, https://lcec-inc.com/"
2022157295,"SIVILTECHNOLOGY, ARCHER STREET","SIVILTECHNOLOGY, ARCHER STREET","ARCHER STREET LLC, SIVIL TECHNOLOGIES INC","ARCHER STREET, nan","4518 11TH AVENUE, PO BOX 561782","LOS ANGELES, LOS ANGELES","tony@archerstreet.com, tony@sivilco.com","https://www.archerstreet.com/, nan"
2085960528,"AJSURVEYING, AJSWIRESAGGING","AJSURVEYING, AJSWIRESAGGING","A J SURVEYING, INC., A J S WIRE SAGGING, INC.","nan, nan","10957 CANIS LN, 9431 DOWDY DR, SUITE 2","SAN DIEGO, SAN DIEGO","nan, nan","nan, nan"
2092290735,"MPSTRUCTURALENGINEER, COVENANT SUPPLY HOUSE, N...","MPSTRUCTURALENGINEER, COVENANT SUPPLY HOUSE, N...","MP STRUCTURAL ENGINEERS, NDS CONSULTING LLC, E...","nan, nan, COVENANT SUPPLY HOUSE","2633 CELAYA CIR, 22320 FOOTHILL BLVD, SUITE 60...","SAN RAMON, HAYWARD, STOCKTON","davis@mpstructural.com, davis@ndsconsultingllc...","http://www.mpstructural.com/, nan, http://www...."
...,...,...,...,...,...,...,...,...
9519927316,VONDEZ,"VONDEZ, VONDEZ LLC","VONDEZ LLC, A I CONSTRUCTION","nan, VONDEZ LLC","nan, 16427 SADDLEBROOK LN","MORENO VALLEY, MORENO VALLEY","vondezllc@yahoo.com, nan","nan, nan"
9708468145,"THRIVISTER, RAZCOACHINGSVC","THRIVISTER, RAZCOACHINGSVC","THRIVISTER LLC,, RAZ COACHING SERVICES LLC","nan, nan","nan, 109 NORTH GRANITE CIRCLE","FOLSOM, FOLSOM","michelle@thrivister.com, michelle@razcoaching.com","nan, htttp://www.razcoaching.com"
9728280711,WESTERN REFINING RETAIL LLC,WESTERN REFINING RETAIL LLC,"SPEEDWAY #5450, SPEEDWAY #4608","WESTERN REFINING RETAIL LLC, WESTERN REFINING ...","9490 MIRA MESA BLVD, 2435 OTAY CENTER DR","SAN DIEGO, SAN DIEGO","nan, nan","nan, nan"


In [13]:
reordered_df = reordered_df.reset_index()
reordered_df 

,Phone,MTAuID_Unique,MTAuID_Correct,Name,DBA Name,Address,City,Email,Website
0,16122101599,MARYBACONINTERIOR,"MARYBACONINTERIOR, MARYBACONART","MARY BACON ART, MARY BACON INTERIORS","nan, nan","1001 FRONT ST., 1001 FRONT ST","CRESCENT CITY, CRESCENT CITY","nan, nan","https://marybaconart.com/, http://www.marybaco..."
1,17603538110,"DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT","DUBOSEDESIGNGROUP, LCENGINEERINGCONSULTANT","DUBOSE DESIGN GROUP, INC., LC ENGINEERING CONS...","nan, nan","1065 W STATE STREET, 1065 W STATE STREET","EL CENTRO, EL CENTRO","tomdubose@sbcglobal.net, carloscorrales@dde-in...","dubosedesigngroup.com, https://lcec-inc.com/"
2,2022157295,"SIVILTECHNOLOGY, ARCHER STREET","SIVILTECHNOLOGY, ARCHER STREET","ARCHER STREET LLC, SIVIL TECHNOLOGIES INC","ARCHER STREET, nan","4518 11TH AVENUE, PO BOX 561782","LOS ANGELES, LOS ANGELES","tony@archerstreet.com, tony@sivilco.com","https://www.archerstreet.com/, nan"
3,2085960528,"AJSURVEYING, AJSWIRESAGGING","AJSURVEYING, AJSWIRESAGGING","A J SURVEYING, INC., A J S WIRE SAGGING, INC.","nan, nan","10957 CANIS LN, 9431 DOWDY DR, SUITE 2","SAN DIEGO, SAN DIEGO","nan, nan","nan, nan"
4,2092290735,"MPSTRUCTURALENGINEER, COVENANT SUPPLY HOUSE, N...","MPSTRUCTURALENGINEER, COVENANT SUPPLY HOUSE, N...","MP STRUCTURAL ENGINEERS, NDS CONSULTING LLC, E...","nan, nan, COVENANT SUPPLY HOUSE","2633 CELAYA CIR, 22320 FOOTHILL BLVD, SUITE 60...","SAN RAMON, HAYWARD, STOCKTON","davis@mpstructural.com, davis@ndsconsultingllc...","http://www.mpstructural.com/, nan, http://www...."
...,...,...,...,...,...,...,...,...,...
3408,9519927316,VONDEZ,"VONDEZ, VONDEZ LLC","VONDEZ LLC, A I CONSTRUCTION","nan, VONDEZ LLC","nan, 16427 SADDLEBROOK LN","MORENO VALLEY, MORENO VALLEY","vondezllc@yahoo.com, nan","nan, nan"
3409,9708468145,"THRIVISTER, RAZCOACHINGSVC","THRIVISTER, RAZCOACHINGSVC","THRIVISTER LLC,, RAZ COACHING SERVICES LLC","nan, nan","nan, 109 NORTH GRANITE CIRCLE","FOLSOM, FOLSOM","michelle@thrivister.com, michelle@razcoaching.com","nan, htttp://www.razcoaching.com"
3410,9728280711,WESTERN REFINING RETAIL LLC,WESTERN REFINING RETAIL LLC,"SPEEDWAY #5450, SPEEDWAY #4608","WESTERN REFINING RETAIL LLC, WESTERN REFINING ...","9490 MIRA MESA BLVD, 2435 OTAY CENTER DR","SAN DIEGO, SAN DIEGO","nan, nan","nan, nan"
3411,9739798198,H&M FASHION USA INC,"H&M FASHION USA INC, HMFASHIONUSA","H&M, H&M FASHION USA INC","H&M FASHION USA INC, nan","4461 CAMINO DE LA PLAZA SUITE 301, 7007 FRIARS...","SAN YSIDRO, SAN DIEGO","nan, nan","nan, nan"


# Step 7:
- Download the new Data into a csv file

In [14]:
reordered_df.to_csv('Companies_samephone.csv', index=False, encoding='utf-8')

In [15]:
FileLink('Companies_samephone.csv')

/Users/stevenyu/personal_projects/MASON-TILLMAN-ASSOCIATES-LTD-main/Companies_samephone.csv